In [1]:
from bs4 import BeautifulSoup
from scipy.io import loadmat
import cv2
import numpy as np
import os
import requests
import csv

In [2]:
labels = [i - 1 for i in loadmat('./imagelabels.mat')['labels'][0].tolist()]
list_dirs = sorted(os.listdir('./102flowers/jpg'))
img_to_lbl = dict(zip(list_dirs, labels))

In [3]:
response = requests.get('https://www.robots.ox.ac.uk/~vgg/data/flowers/102/categories.html')
print(response.ok, response.elapsed)
soup = BeautifulSoup(response.text, 'html.parser')

True 0:00:01.286077


In [4]:
result = {}
table = soup.table
body = table.tbody
cnt = 0
for tr in body.find_all('tr'):
    imL, imC, imR = None, None, None
    lblL, lblC, lblR = None, None, None
    for td in tr.find_all('td'):
        if td.get('class')[0] == 'imL': 
            imL = td.img.get('src')
            imL = 'image_' + imL.split('/')[-1].split('_')[-1]
        if td.get('class')[0] == 'imC':
            imC = td.img.get('src')
            imC = 'image_' + imC.split('/')[-1].split('_')[-1]
        if td.get('class')[0] == 'imR':
            imR = td.img.get('src')
            imR = 'image_' + imR.split('/')[-1].split('_')[-1]

        if 'labL' in td.get('class'):
            lblL = td.text

        if 'labC' in td.get('class'):
            lblC = td.text

        if 'labR' in td.get('class'):
            lblR = td.text

    lbl_numL = img_to_lbl[imL]
    lbl_numC = img_to_lbl[imC]
    lbl_numR = img_to_lbl[imR]
    result[lbl_numL] = lblL
    result[lbl_numC] = lblC
    result[lbl_numR] = lblR

In [5]:
with open('label_names.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(['Label', 'Name'])
    for k, v in result.items():
        writer.writerow([k, v])

## Create a Tensorflow Lite (TFLITE) output

In [6]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [9]:
import tensorflow as tf
from tensorflow import keras
from keras.applications.efficientnet_v2 import preprocess_input

In [10]:
model = tf.keras.models.load_model('./saves/tf/02_efficient_netB0_97.tf/')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-b0 (Functio  (None, 7, 7, 1280)        5919312   
 nal)                                                            
                                                                 
 conv2d (Conv2D)             (None, 7, 7, 1024)        11797504  
                                                                 
 batch_normalization (Batch  (None, 7, 7, 1024)        4096      
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 7, 7, 1024)        0         
                                                                 
 global_average_pooling2d (  (None, 1024)              0     

In [11]:
predict_model = keras.Sequential([
    keras.layers.Lambda(lambda x: preprocess_input(x)),
    model,
    keras.layers.Activation('softmax')
])

In [12]:
img = cv2.imread('./102flowers/jpg/image_00001.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224, 224))[None, ...]
pred = predict_model(tf.cast(img, tf.float32))
print(f'%{np.max(pred):.3f} {result[np.argmax(pred)]}')

%1.000 passion flower


In [53]:
predict_model.save('./saves/temp/mobilenet_0.97.tf')

INFO:tensorflow:Assets written to: ./saves/temp/mobilenet_0.97.tf/assets


INFO:tensorflow:Assets written to: ./saves/temp/mobilenet_0.97.tf/assets


In [54]:
converter = tf.lite.TFLiteConverter.from_saved_model('./saves/temp/mobilenet_0.97.tf/')
tflite = converter.convert()
with open('./saves/tflite/mobilenet_0.97.tflite', 'wb') as file:
    file.write(tflite)

Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 62, Total Ops 102, % non-converted = 60.78 %
 * 62 ARITH ops

- arith.constant:   62 occurrences  (f32: 60, i32: 2)



  (f32: 15)
  (f32: 13)
  (f32: 1)
  (f32: 1)
  (f32: 1)
  (f32: 4)
  (f32: 1)
  (f32: 1)


In [55]:
!rm -rf ./saves/temp